### DTS - Complete Streets

# 08 - Modal Priortization

**Author:** rmangan

---

**Purpose:**

Implement Modal Priortizaiton logic for street segments that have Unconstrained Modal Width > ROW Width estimate. 

**This script performs the following functions:**

1. Pre-process data - add all necessary fields, copy unconstrained values to new constrained fields

2. Process Boulevards

3. Process Avenues

4. Process Main Streets

5. Process Major Streets

6. Process Streets/Lanes/Alleys

7. Cleanup (if needed)

8. Results Analysis

9. Join Data to Feature Services (if needed)

3. (optional utility functions) Resent all typologies to NULL for reprocessing


**Global Assumptions and Notes:**
1. All typology comments with values in "Note" field are skipped, comments must be manually addressed in ArcGIS Pro.
2. item
3. item
4. item

**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - used for common data geo-processing
2. arcgis 1.83 - used for connecting to AGOL org and related items
2. pandas 1.1 - (optional) used for QC purposess and data exploration

In [ ]:
# import modules
import arcpy
import os
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS

In [ ]:
from IPython.display import Image

In [ ]:
#test image embed for documentation purposes

Image("https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/45a7bf56620b99717ae38310a31b367948880137/Boulevard.jpg")


In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\modal\Modal_Composite4.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# (optional) login to AGOL if accessing feature services
my_gis = GIS()

print("ArcGIS Online Org account")

pw = input("Enter AGOL password...")

gis = GIS("http://dithnl.maps.arcgis.com", "ray.mangan_ch2m", pw)
print("Logged in as " + str(gis.properties.user.username))

In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

modal_composite_url = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"

modal_composite_url_copy = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"


In [ ]:
#code planning

#ensure using latest modal priority dataset

#01 - add necessary fields

#boulevard

#avenue

#main street

#major street

#street/lane/alley

#cleanup

#results analysis

#join data to feature service (may be manual step)


In [ ]:
#Calculate Constrained Modal width

def calc_constrained():
    #calculate all constrained modal width fields
    
    count = 0
    err_count = 0

    fields = ["SEGMENTID",                 #0
              "modal_width_ped_const",     #1
              "modal_width_bike_const",    #2
              "modal_width_auto_const",    #3
              "modal_width_bus_const",     #4
              "modal_width_park_const",    #5
              "modal_width_medians_const", #6
              "modal_width_const",         #7
              "modal_area_const",          #8
              "modal_area_diff_const",     #9
              "Fac_Type_BP",               #10
              "Fac_Type_BR",               #11
              "NewTier"]                   #12
    
    where_clause = ('"cs_type"= \'{}\''.format(cs_type))

    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            #print("row")
            try:
                #igonore shared road-way widths from bike width
                if row[10] == "Shared Roadway" or row[11] == "Shared Roadway":     
                    width = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                    row[7] = width
                    count +=1
                
                #ignore bike shared use path for ped > 7    
                elif (row[10] == "Shared Use Path" or row[10] == "Shared Use Path") and row[1] is not None and row[1] > 7:                   
                    width = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                    row[7] = width
                    count +=1
                
                #if shared use path is proposed on segment where future ped width is less than 7, assume 7 for ped width in calc   
                elif (row[10] == "Shared Use Path" or row[11] == "Shared Use Path") and row[1] is not None and row[1] < 7:   
                    width = sum(filter(None,(row[1], 7,  row[3], row[4], row[5], row[6])))
                    row[7] = width
                    count +=1
            
                #summarize all other segments
                else:  
                    width = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
                    row[7] = width
                    count +=1
            
                cursor.updateRow(row)
            
            except (ValueError,TypeError) as error:
                err_count += 1
                print("Error at SegmentID: {0} - {1} ".format(row[0],error))
                
    print("Constrained widths calculated for {0} items".format(count))
    print("Errors (total) for {0} items".format(err_count))
    print("Modal Widths - Constrained complete.")

In [ ]:
#Remove parkign from Constrained Street Types if available, only alters Constrained Modal Width fields.
#Note this function does NOT recalculate modal width totals

def remove_parking(cs_type):
    #set constrained modal values for parking to 0 for a given street type
    #accept cs_type as an input street type variable
    
    count = 0
           
    #fields to expose to update cursor            
    fields = ["modal_width_park_const",   #0
              "modal_width_const"]        #1  
        
    where_clause = ('"cs_type"= \'{}\' AND' 
                    '"modal_width_park_const" > 0 AND'
                    '(NOT "modal_width_park_const" IS NULL) AND'
                    '("modal_width_const" > "mean_row_2020")'.format(cs_type))
        
    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            try:
                row[0] = 0    
                count += 1
                cursor.updateRow(row)
            except ValueError as error:
                print(error)
                        
    print("Done - {0} records updated".format(count))

In [ ]:
def reduce_bike(cs_type):
    #Reduce Proposed Bike widths to minimum if present, set parallel route evaluation flag to 1
    #Only processes streets where parking is not availalble (see where clause in update cursor)
    #Does not reduce existing Bike Facilities

    count = 0
    
    #fields to expose to update cursor
    fields = ["Fac_Type_BP",            #0
              "Fac_Type_BR",            #1
              "modal_width_bike_const", #2
              "bike_parallel_route"]    #3
         
    where_clause = ('"cs_type" = \'{0}\' AND' 
                    '("modal_width_park_const" = 0 OR ("modal_width_park_const" IS NULL)) AND (NOT ("Fac_Type_BP" IS NULL AND "Fac_Type_BR" IS NULL)) AND'
                    '("modal_width_const" > "mean_row_2020")'.format(cs_type))
    
    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            #print("row")
            try:
                #Buffered Bike Lanes - Set to minimum 8' width and flag for parallel route evaluation
                if (row[0] == "Buffered Bike Lane" or 
                    row[1] == "Buffered Bike Lane"):
                    row[2] = 8
                    row[3] = 1
                
                #Protected Bike Lanes - Set to minimum 10' width and flag for parallel route evaluation 
                elif (row[0] == "Protected Bike Lane" or
                      row[1] == "Protected Bike Lane"):
                    row[2] = 10
                    row[3] = 1
            
                #All other bike features - Set to 5' width and flag for parallel route evaluation
                elif (row[0] == "Bike Lane" or
                      row[0] == "Climbing Lane" or
                      row[0] == "Shoulder Bikeway" or
                      row[0] == "Shared Use Path" or
                      row[0] == "Shared Roadway" or
                      row[1] == "Bike Lane" or
                      row[1] == "Climbing Lane" or
                      row[1] == "Shoulder Bikeway" or
                      row[1] == "Shared Use Path" or
                      row[1] == "Shared Roadway"):
                    row[2] = 5
                    row[3] = 1
                
                cursor.updateRow(row)
                count += 1
            
            except ValueError as error:
                print(error)

    print("Done - {0} records updated".format(count))      

In [ ]:
def cs_flag(cs_type):
#Set Street Type re-evaluation flag for constrained streets
         
    count = 0

    #fields to expose to update cursor
    fields = ["cs_type_revise"] #0
    
    where_clause = ('"cs_type" = \'{0}\' AND' 
                    '("modal_width_const" > "mean_row_2020")'.format(cs_type))

    with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
        for row in cursor:
            try:
                row[0] = 1   
                cursor.updateRow(row)
                count += 1
            
            except ValueError as error:
                print(error)
                                          
    print("Done - {0} records updated".format(count))

## 01 - Pre-process data
---

Add all necessary fields for data procesing. Copy unconstrained modal width and area values into newly added constrained modal fields. Constrained fields will then be updated by subsquent processing sections. This will enable comparison between Unconstrained and Constrained modal values.

**This section performs the following functions:**

1. Add all necessary fields
2. Copy unnconstrained modal values into new constrained fields (to be updated in later sections)

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
#01 - add necessary fields & write all modal values to new fields
print("Adding new fields...")
#store field, field alias in dict
new_fields = {"modal_width_ped_const":      "Modal Width - Ped Constrained",
              "modal_width_bike_const":     "Modal Width - Bike Constrained",
              "modal_width_auto_const":     "Modal Width - Auto Constrained",
              "modal_width_bus_const":      "Modal Width - Bus Constrained",
              "modal_width_park_const":     "Modal Width - Parking Constrained",
              "modal_width_medians_const":  "Modal Width - Medians Constrained",
              "modal_width_const":          "Modal Width - Total Constrained",
              "modal_area_const":           "Modal Area - Total Constrained (sq. ft.)",
              "modal_area_diff_const":      "Modal Area - Total Constrained Exceedence (sq. ft.)",
              "bike_parallel_route":        "Bike - Evaluate Parallel Routes",
              "lane_count_rev":             "Lane Count (Constrained)",
              "cs_type_revise":             "Complete Street Type - Re-Evaluate"}

#get fields in current input datset
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

#loop through dict and add fields if they are not currently in the dataset
for key,value in new_fields.items():
    if key in field_names:
        print("{0} field already present, skipping".format(key))
    elif key in ("modal_area_const","modal_area_diff_const"):
        print("Adding field {0}".format(key))
        arcpy.AddField_management(modal_composite,field_name=key, field_type="FLOAT", field_alias = value)
    else:
        print("Adding field {0}".format(key))
        arcpy.AddField_management(modal_composite,field_name=key, field_type="SHORT", field_alias = value)
          
print("Field additions complete")
print("\n")

In [ ]:
#write all previous proposed modal values to new fields (values will be altered in subsquent processing)

#fields to expose to update cursor
fields = ["modal_width_auto_prop",    #0
          "modal_width_auto_const",   #1
          "modal_width_bike_prop",    #2
          "modal_width_bike_const",   #3
          "modal_width_bus_prop",     #4
          "modal_width_bus_const",    #5
          "modal_width_park_prop",    #6
          "modal_width_park_const",   #7
          "modal_width_ped_prop",     #8
          "modal_width_ped_const",    #9
          "modal_width_max",          #10
          "modal_width_const",        #11
          "modal_area_max" ,          #12     
          "modal_area_const",         #13
          "modal_area_diff",          #14
          "modal_area_diff_const"]    #15

print("Copying all Unconstrained values to Constrained fields for initial processing...")
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:            
            row[1] = row[0]
            row[3] = row[2]
            row[5] = row[4]
            row[7] = row[6]
            row[9] = row[8]
            row[11] = row[10]
            row[13] = row[12]
            row[15] = row[14]         

            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            print(error)
            #err_count += 1
            #print("Error at SegmentID: {0} - {1} ".format(row[0],error))          

print("Done - Constrained Fields calculated from Unconstrained")

## 02 - Modal Priortization - Boulevards
---

Calculate Modal Priortization logic and revised Modal widths for Boulevards.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/45a7bf56620b99717ae38310a31b367948880137/Boulevard.jpg" alt="Logic_Boulevard" style="height: 200px;"/>

**This section performs the following functions:**

1. If Street Type = Boulevard, ROW = Constrained, and parking is available, remove parking
2. Re-calcluate Constrained Modal width field
3. If Street Type = Boulevard, ROW = Constrained, and parking = 0 or NULL, and proposed bike features are present, reduce bike widhts to minimum and set parallel route flag to 1
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Minimum Modal widths for Bike Facilities used:
   1. Buffered Bike Lane - 8'
   2. Protected Bike Lane - 10'
   3. Bike Lane, Climbing Lane, Shoulder Bikeway, Shared Use Path, Shared Roadway - 5'

In [ ]:
## 01 - remove parking

print("Processing Boulevards....")
print("02.01 - removing parking...\n")
remove_parking("Boulevard")
print("02.01 - done\n")

In [ ]:
## 02 - Re-Calcualte Constrained Modal Width

print("02.02 - Re-calculate constrained width values\n")
calc_constrained("Boulevard")
print("02.02 - done\n")

In [ ]:
## 03 - Reduce Bike widths to minimum if present, set parallel route evaluation flag to 1

print("02.03 - Reduce proposed bike facilities for remaining constrained boulevards, flag for parallel route evaluation\n")  
reduce_bike("Boulevard")
print("02.03 - done\n")

In [ ]:
## 04 - Re-Calcualte Constrained Modal Width for Boulevards

print("02.04 - Re-calculate constrained width values\n")
calc_constrained("Boulevard")
print("02.04 - done\n")

In [ ]:
## 05 - Set all remaining constrained widths that are boulevards to street type re-evaluation

print("02.05 - Set street type re-evaluation flage for remaining constrained boulevards\n")
cs_flag("Boulevard")
print("02.05 - done\n")

print("Boulevards Complete")

## 03 - Modal Prioritization - Avenues
---

Calculate Modal Priortization logic and revised Modal widths for Boulevards.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/Avenue.jpg" alt="Logic_Avenue" style="height: 200px;"/>

**This section performs the following functions for Avenues:**

1. If Street Type = Avenue, ROW = Constrained, and parking is available, remove parking
2. Re-calcluate Constrained Modal width field
3. If Street Type = Avenue, ROW = Constrained, and parking = 0 or NULL, and VOC <= 0.8, reduce auto transit lane count.
4. Re-calcualte Constrained Modal width field
5. If Street Type = Avenue, ROW = Constrained, parking = 0 or NULL, VOC > 0.8, reduce sidewalk facilities to minimum width
6. Re-calcualte Constrained Pedestrian width
7. Re-calculalate Constrained Modal width fields
8. If still constrained, set flag for street type re-evaluation


**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Minimum Modal widths for Pedestiran facilities used:
   1. X - y'
   2. X - y'
   3. X - y'

In [ ]:
## 01 - Remove parking

print("Processing Boulevards....\n")

print("03.01 - removing parking...\n")
remove_parking("Avenue")
print("03.01 - done\n")

In [ ]:
## 02 - Re-Calculate Constrained Modal Width

print("03.02 - Re-calculate constrained width values\n")
calc_constrained("Avenue")
print("03.02 - done\n")

In [ ]:
## 03 - Remove transit lane if VOC <= 0.8

print("03.03 - Re-calculate constrained width values\n")

#Reduce lane count by 1 for all segments

#Re-calculate modal with for auto using new lane counts

print("03.03 - done\n")






In [ ]:
## 04 - Re-Calcualte Constrained Modal Width

print("03.04 - Re-calculate constrained width values\n")
calc_constrained("Avenue")
print("03.04 - done\n")

In [ ]:
## 05 - Reduce sidewalk widths to minimum

In [ ]:
## 06 - Re-Calcualte Constrained Modal Width

print("03.06 - Re-calculate constrained width values\n")
calc_constrained("Avenue")
print("03.06 - done\n")

In [ ]:
## 07 - Set all remaining constrained widths that are boulevards to street type re-evaluation

print("03.07 - Set street type re-evaluation flage for remaining constrained segments\n")
cs_flag("Avenue")
print("03.07 - done\n")

print("Avenues Complete")

## 04 - Modal Priortization - Main Streets
---

Calculate Modal Priortization logic and revised Modal widths for Main Strets.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/MainStreet.jpg" alt="Logic_Avenue" style="height: 200px;"/>

**This section performs the following functions for all Main Streets:**

1. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
2. Re-calcluate Constrained Modal width field
3. If ROW = Constrained, and parking > 0 or NULL, remove parking
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
## 01 - Combine Bike & Pedestrian Facilities

print("Processing Main Streets....\n")

print("04.01 - Combining Bike & Pedestrian facilities...\n")






print("04.01 - done\n")

In [ ]:
## 02 - Re-Calculate Constrained Modal Width

print("04.02 - Re-calculate constrained width values\n")
calc_constrained("Main Street")
print("04.02 - done\n")

In [ ]:
## 03 - Remove parking

print("Processing Main Streets....\n")

print("04.03 - removing parking...\n")
remove_parking("Main Street")
print("04.03 - done\n")

In [ ]:
## 04 - Re-Calculate Constrained Modal Width

print("04.04 - Re-calculate constrained width values\n")
calc_constrained("Main Street")
print("04.04 - done\n")

In [ ]:
## 05 - Set all remaining constrained widths to street type re-evaluation

print("04.05 - Set street type re-evaluation flage for remaining constrained segments\n")
cs_flag("Main Street")
print("04.05 - done\n")

print("Main Streets Complete")

## 05 - Modal Priortization - Major Streets
---

Calculate Modal Priortization logic and revised Modal widths for Major Streets.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/MajorStreet.jpg" alt="Logic flow diagram for Major Streets" style="height: 200px;"/>

**This section performs the following functions for all Major Streets:**

1. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
2. Re-calcluate Constrained Modal width field
3. If ROW = Constrained, and parking > 0 or NULL, remove parking
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Logic is identical to logic used for Boulevards but maintained as seperate code sections to allow for individual changes if needed
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
## 01 - Remove parking

print("Processing Major Streets....\n")

print("05.01 - removing parking...\n")
remove_parking("Major Street")
print("05.01 - done\n")

In [ ]:
## 02 - Re-Calculate Constrained Modal Width

print("05.02 - Re-calculate constrained width values\n")
calc_constrained("Major Street")
print("05.02 - done\n")

In [ ]:
## 03 - Reduce Bike widths to minimum if present, set parallel route evaluation flag to 1

print("05.03 - Reduce proposed bike facilities for remaining constrained boulevards, flag for parallel route evaluation\n")  
reduce_bike("Major Street")
print("05.03 - done\n")

In [ ]:
## 04 - Re-Calculate Constrained Modal Width

print("05.04 - Re-calculate constrained width values\n")
calc_constrained("Major Street")
print("05.04 - done\n")

In [ ]:
## 05 - Set all remaining constrained widths to street type re-evaluation

print("02.05 - Set street type re-evaluation flage for remaining constrained boulevards\n")
cs_flag("Boulevard")
print("02.05 - done\n")

print("Major Streets Complete")

## 05 - Modal Priortization - Streets, Lanes, Alleys
---

Calculate Modal Priortization logic and revised Modal widths for Main Strets.

<img src="https://raw.githubusercontent.com/Ray-Mangan/DTS-Complete-Streets/main/Street.jpg" alt="Logic_Avenue" style="height: 200px;"/>

**This section performs the following functions for all Main Streets:**

1. If ROW = Constrained, and bike facilities are proposed, combine bike & ped into shared area
2. Re-calcluate Constrained Modal width field
3. If ROW = Constrained, and parking = 0 or NULL, and proposed bike features are present, reduce bike widhts to minimum and set parallel route flag to 1
4. Re-calcualte Constrained Modal width field
5. If still constrained, set flag for street type re-evaluation

**Assumptions and Notes:**
1. Assumpmtion with a [LINK](https://honolulu-cchnl.opendata.arcgis.com/datasets/8068469b47834d3ca4bc299d4079f35f_0/explore?location=21.483400,-157.964050,11.24)
2. Existing Bike Facilites will not be reduced in size
   1. **Nested Item** - and a nested description

In [ ]:
#get fields in current input datset
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

"SEGMENTID" in field_names

const_fields = ["SEGMENTID",                 #0
                "modal_width_ped_const",     #1
                "modal_width_bike_const",    #2
                "modal_width_auto_const",    #3
                "modal_width_bus_const",     #4
                "modal_width_park_const",    #5
                "modal_width_medians_const", #6
                "modal_width_const",         #7
                "modal_area_const",          #8
                "modal_area_diff_const"      #9
                "Fac_Type_BP",               #10
                "Fac_Type_BR",               #11
                "NewTier"]                   #12

for new_fields in const_fields:
    print(new_fields in field_names)

In [ ]:
#Calculate Constrained Modal metrics
const_fields = ["SEGMENTID",                 #0
                "modal_width_ped_const",     #1
                "modal_width_bike_const",    #2
                "modal_width_auto_const",    #3
                "modal_width_bus_const",     #4
                "modal_width_park_const",    #5
                "modal_width_medians_const", #6
                "modal_width_const",         #7
                "modal_area_const",          #8
                "modal_area_diff_const"      #9
                "Fac_Type_BP",               #10
                "Fac_Type_BR",               #11
                "NewTier"]                   #12      
    
count = 0
err_count = 0

print("Calculating Constrained fields..")

with arcpy.da.UpdateCursor(modal_composite, const_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #igonore shared road-way widths from bike width
            if row[10] == "Shared Roadway" or row[11] == "Shared Roadway":     
                width = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                row[7] = width
                count +=1
                
            #ignore bike shared use path for ped > 7    
            elif (row[10] == "Shared Use Path" or row[10] == "Shared Use Path") and row[1] is not None and row[1] > 7:                   
                width = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                row[7] = width
                count +=1
                
            #if shared use path is proposed on segment where future ped width is less than 7, assume 7 for ped width in calc   
            elif (row[10] == "Shared Use Path" or row[11] == "Shared Use Path") and row[1] is not None and row[1] < 7:   
                width = sum(filter(None,(row[1], 7,  row[3], row[4], row[5], row[6])))
                row[7] = width
                count +=1
            
            #summarize all other segments
            else:  
                width = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
                row[7] = width
                count +=1
            
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
                
print("Constrained widths calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Constrained complete.")

In [ ]:
#pd.set_option('display.max_rows', 250)
sdf.loc[filt2,fields].sort_values(by="FULLNAME")

In [ ]:
#Create DF for CSDM analysis
modal_df = pd.DataFrame.spatial.from_featureclass(modal_composite)

modal_df_fields = modal_df.columns.values.tolist()

modal_df_fields.sort()

#print all fields for reference
for i in modal_df_fields:
    print(i)

In [ ]:
#Fields for Dataframe
df_fields = ["SEGMENTID",
             "FULLNAME",
             "CS_type",
             "ONEWAY",
             "mean_row_2020",
             "row_exist_num",
             "row_prop_num",
             "modal_width_park_exist",
             "modal_width_bike_exist",
             "modal_width_bike_prop",
             "VOC_max",
             "modal_width_max"]

#Pre-set Filters for Dataframe

filt_boulevard = ((modal_df["CS_type"] == "Boulevard" ))

filt_avenue = ((modal_df["CS_type"] == "Avenue" ))

filt_main_street = ((modal_df["CS_type"] == "Main Street" ))

filt_street = ((modal_df["CS_type"] == "Street" ))

filt_constrained = ((modal_df["mean_row_2020"] < modal_df["modal_width_max"] ))


filt_parking = ((modal_df["modal_width_park_exist"] > 0))

filt_bike = ((modal_df["modal_width_bike_prop"] > 0))

filt_VOC = ((modal_df["VOC_max"] > 0.8))

filt_row = ((modal_df["mean_row_2020"] < 70 ))

filt_cs_examples = ((~modal_df["CS_example_num"].isnull()))



filt_example = (((modal_df["AB_SPEED"] > 35 ) |
             (modal_df["BA_SPEED"] > 35 )) &
             (modal_df["TOT_FLOW_D"] > 10000) &
             ((modal_df["AB_LANE"] + modal_df["BA_LANE"]) >= 4 )
             & ((~modal_df["CS_example_num"].isnull()))  
            )

filt_change = (((modal_df["AB_SPEED"] < 35 ) |
             (modal_df["BA_SPEED"] < 35 )) &
             (modal_df["TOT_FLOW_D"] > 10000) &
             ((modal_df["AB_LANE"] + modal_df["BA_LANE"]) <= 4 )
             #& ((~cs_df["CS_example_num"].isnull()))  
            )

In [ ]:
#pd.set_option('display.max_rows', 250)

#cs_df.loc[filt35_CS,df_fields].sort_values(by="FULLNAME")

#cs_df.filter(df_fields)

#Parking
modal_df.loc[(filt_parking & filt_boulevard & filt_constrained), df_fields]

#Boulevards
#modal_df.loc[(modal_df["CS_type"] == "Boulevard"),df_fields]

#Avenues
#modal_df.loc[(modal_df["CS_type"] == "Avenue"),df_fields]

#Main Streets
#modal_df.loc[(modal_df["CS_type"] == "Main Street"),df_fields]


#modal_df.loc[filt_constrained, df_fields]

In [ ]:
#CSDM Street Typology Pre-Processing - REMOVE FROM FINAL CODE


#join CSDM exmaple values into dataset
#join fields from median classification
print("Joining CSDM Example field...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = modal_composite_02
join_table_field = "SEGMENTID"
join_fields = ["CSDM_EXAMPLE"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")
    
print("Done")


#alter joined field to match naming conventiones
arcpy.management.AlterField(modal_composite_05a,"CSDM_EXAMPLE", new_field_name = "CS_example_num", new_field_alias = "Complete Streets - Example Number" )

#add CS example type field
arcpy.AddField_management(modal_composite_05a,field_name="CS_example_type",field_type="TEXT",field_alias = "Complete Streets - Example Type", field_length = 30)

#add CS type field
arcpy.AddField_management(modal_composite_05a,field_name="CS_type",field_type="TEXT",field_alias = "Complete Streets - Type", field_length = 30)




In [ ]:
#CSDM Street Typology Processing

#Set CSDM analysis fields and filters

#fields to expose to update cursor
cs_fields = ["AB_LANE",               #0
             "BA_LANE",               #1
             "lanes_assumed",         #2
             "AB_SPEED",              #3
             "BA_SPEED",              #4
             "mean_row_2020",         #5
             "median_type",           #6
             "median_width",          #7
             "TOT_FLOW_D",            #8
             "CS_type",               #9
             "SEGMENTID",             #10
             "FULLNAME",              #11
             "length_urban_percent" , #12     
             "length_ag_percent",     #13
             "aadt",                  #14
             "SPD_LIMIT"]             #15

count_blvd = 0
count_ave = 0
count_main = 0
count_major_st = 0
count_st = 0
count_lane = 0
count_mall = 0
count_rural = 0
count_scenic = 0
count_error = 0

#"{0},{1}".format(round(float([AB_SPEED]),0),round(float([BA_SPEED]),0))



print("Setting Street Types....")
count = 0
with arcpy.da.UpdateCursor(modal_composite, cs_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #Set Boulevards
            if (((row[0] is not None) and (row[1] is not None) and row[0] + row[1] >= 4) and #lanes >=4                            
               ((row[15] is not None) and row[15] > 35) or row[6] == "R"): #Speed Limit >35
                
                count_blvd += 1
                row[9] = "Boulevard"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                      
            #Set Main Streets
            elif ((row[15] is not None) and (row[15] <= 35) and #Speed Limit <=35
                  (row[14] is not None and row[14] >= 10000) and #AADT > 10000
                  (row[12] is not None and row[12] > .5)): #Urban Percent > 50%
                
                count_main += 1
                row[9] = "Main Street"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                
            #Set Avenues
            
            
            
            elif ((row[15] is not None) and (row[15] <= 35) and #Speed Limit <=35
                  (row[14] is not None and row[14] >= 10000) and #AADT > 10000
                  ((row[12] is not None and row[12] <= .5) or row[12] is None)): #Urban Percent is None or less than 50%
                
                count_ave += 1
                row[9] = "Avenue"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                    
            #Set Malls - hardcoded based on Segment Ids for Dukes Ln, Fort St Mall, S. Hotel St, Kekaulike St.
            elif (row[10] in (18914, 18967, 18984, 
                              19033, 19090, 19114, 
                              19148, 19152, 19233, 
                              19234, 19271, 19272, 
                              19306, 19319, 19333, 
                              19357, 19358, 19411, 
                              19456)):
                count_mall += 1
                row[9] = "Mall"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
            
            #Set Lanes & Alleys - ROW < 30 and has 2 lanes max and does not have TransCAD listing
            elif ((row[5] is not None and row[5] < 30) and #width < 30
                  (((row[0] is not None) and (row[1] is not None) and (row[0] + row[1] <=2)) or row[0] is None)): #2 lanes max or TransCAD is null 
                count_lane += 1
                row[9] = "Lane/Alley"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
            
            #Set Rural Roads - Rural % > 50 and does not have TransCAD or AADT data
            elif (row[13] is not None and row[13] > .5 and (row[0] is None) and (row[14] is None)):
                count_rural += 1
                row[9] = "Rural Road"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                
            #Set Scenic Byway's - hardcoded
            elif (row[10] in (18589,18648,23714,
                              33873,16814,33532,
                              33533,33534,33535,
                              33536,33537,33538)):
                count_scenic += 1
                row[9] = "Scenic Byway"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                  
            #Set Major Street
            # 7/23/21 - MODIFY TO INCLUDE "SPD_LIMIT > 25 And aadt > 10000 And PPN_Final = 0"
            
            elif ((row[15] is not None) and (row[15] <= 25) and #Speed Limit <=25
                  ((row[0] is not None) and (row[1] is not None) and row[0] + row[1] <= 4) and #lanes <= 4   
                  (row[14] is not None) and row[14] >= 4000 and row[14] <= 10000)): #AADT > 10000 
                
                count_major_st += 1
                row[9] = "Major Street"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                    
                    
            #Set Everything else to Street
            else:
                count_st += 1
                row[9] = "Street"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
            
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            count_error += 1
            
print("Type - Boulevard: {}".format(count_blvd))
print("Type - Avenue: {}".format(count_ave))
print("Type - Main: {}".format(count_main))
print("Type - Mall: {}".format(count_mall))
print("Type - Lane: {}".format(count_lane))
print("Type - Rural: {}".format(count_rural))
print("Type - Scenic Byway: {}".format(count_scenic))
print("Type - Major Street: {}".format(count_major_st))
print("Type - Street: {}".format(count_st))
print("Types set for {} records".format((count_blvd + count_ave + count_main + count_mall + count_lane + count_rural + count_scenic + count_st)))

print("I am Error: {}x".format(count_error))

print("Typology Assignments complete.")


In [ ]:
#reset all CS Street Types to Null

with arcpy.da.UpdateCursor(modal_composite,"CS_type") as cursor:
    for row in cursor:
        #print("row")
        try:
            row[0] = None
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            
print("All CS Types reset to None")
          

In [ ]:
#create num fields for modal width calcualtions

#copy input dataset while testing functions
print("copying modal input for processing...")
modal_processing_temp = arcpy.CopyFeatures_management(modal_composite_04,"modal_composite_04a")
print("done")

#add field for assumed lanes for streets that do not have any lane data from TransCAD
arcpy.AddField_management(modal_composite_05a,field_name="lanes_assumed",field_type="SHORT",field_alias = "Lanes (assumed)")

#store field, field alias in dict
new_fields = {"modal_width_ped_exist":  "Modal Width - Ped (Existing)",
              "modal_width_ped_prop":   "Modal Width - Ped (Proposed)",
              "modal_width_bike_exist": "Modal Width - Bike (Existing)",
              "modal_width_bike_prop":  "Modal Width - Bike (Proposed)",
              "modal_width_auto_exist": "Modal Width - Auto (Existing)",
              "modal_width_auto_prop":  "Modal Width - Auto (Proposed)",
              "modal_width_bus_exist":  "Modal Width - Bus (Existing)",
              "modal_width_bus_prop":   "Modal Width - Bus (Proposed)",
              "modal_width_park_exist": "Modal Width - Parking (Existing)",
              "modal_width_park_prop":  "Modal Width - Parking (Proposed)",
              "modal_width_medians":    "Modal Width - Medians",
              "modal_width_max":        "Modal Width - Total Unconstrained"}

#loop through dict and add fields
for key,value in new_fields.items():
    print("Adding field {0}".format(key))
    arcpy.AddField_management(modal_composite_05a,field_name=key,field_type="SHORT", field_alias = value)
    

          
print("Field additions complete")


### Calculate Modal Widths for Auto Traffic

- Calculate existing modal with for auto traffic using lane count x average lane width into `modal_width_auto_exist`. 

- If VOC Max from TransCAD 2040 > 1.0, assume two-way streets have two additonal lanes added (22' additional width), and one-way streets have one additional lane added (11' additonal width). Proposed width is calcualted into `modal_width_auto_prop`

- If VOC Max from TransCAD 2040 is <1.0, existing width is calculted into proposed width


**Assumptions, Notes, and Parameters**
1. Average lane width parameter =  11' wide.
2. Street segments with no lane data have the following additional assumptions applied, in order of implementation: 
    1. If `ONEWAY = 0` (street is two-way) and `mean_row < 100` assume 2 lanes total for line segment. Adjust with `param_two_way`
    2. If `ONEWAY = 0` (street is two-way) and `mean_row > 100` disregard (segments are typically edge cases)
    3. If `ONEWAY IN (1,2)` (street is one-way) and `mean_row < 75` assume 1 lane total for line segment. Adjust with `param_one_way`
    4. If `ONEWAY IN (1,2)` (street is one-way) and `mean_row > 75` disregard (segments are typically edge cases)
3. All street segments that have lane data, but do not have TransCAD VOC Max data have `modal_width_auto_exist` calculated into `modal_width_auto_prop`
4. Remaining street segments with no lane data, no mean_row calcualtion, and no TransCAD data, but have `CLASS = A41` are assumed to be 2 lanes, 22' width (regardless of oneway/two-way)
4. All street segments that were unable to have mean_row calculated, and do not have lane data are ignored


In [ ]:
#Conditional Re-Write
#PARAMETERS
param_one_way = 75.0
param_two_way = 100.0

#fields to expose to update cursor
VOC_fields = ["AB_VOC_EA_1", #0
              "BA_VOC_EA_1", #1
              "AB_VOC_AM_1", #2
              "BA_VOC_AM_1", #3
              "AB_VOC_MD_1", #4
              "AB_VOC_MD_1", #5
              "AB_VOC_PM_1", #6
              "BA_VOC_PM_1", #7
              "AB_VOC_EV_1", #8
              "BA_VOC_EV_1", #9
              "ONEWAY",      #10
              "modal_width_auto_exist", #11
              "modal_width_auto_prop",  #12
              "AB_LANE",     #13
              "BA_LANE",     #14
              "SEGMENTID",   #15
              "ln_exist_num",#16
              "AB_VOC_MAX",  #17
              "BA_VOC_MAX",  #18 
              "mean_row_2020", #19
              "lanes_assumed", #20,
              "CLASS"         #21
             ]

print("Update rows for Traffic widths...")
count = 0
count_01_01 = 0
count_01_02 = 0
count_01_03 = 0
count_01_04 = 0
count_02_01 = 0
count_02_02 = 0
count_02_03 = 0
count_02_04 = 0
count_03_01 = 0
count_03_02 = 0
count_03_03 = 0
count_03_04 = 0
count_03_05 = 0
voc_above = 0
voc_below = 0
voc_null = 0
err_count = 0

with arcpy.da.UpdateCursor(modal_composite, VOC_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #CONDITIONAL GROUP 01
            
            #VOC > 1.0
            #One Way Streets
            #TransCAD Lanes
#             if (not no(row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and #VOC Max > 1.0
#                 ((row[13] is not None or row[14] is not None) and row[10] in [1,2])):
            
            if ((row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and #VOC Max > 1.0
                ((row[13] is not None or row[14] is not None) and row[10] in [1,2])):
                
                #take max of the two lane values and multiply by 11
                current_width = (max(row[13],row[14]))*11
                expanded_width = ((max(row[13],row[14])+1)*11)
            
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                voc_above +=1 
                count_01_01 +=1
                cond = "01_01"
            
            #VOC > 1.0
            #One Way Streets
            #ASSUMED Lanes
            elif ((row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and 
                  ((row[13] is None and row[14] is None and (row[10] in [1,2]) and (row[19] is not None) and row[19] < param_one_way))):
                
                current_width = 11
                expanded_width = 22
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 1
                voc_above +=1 
                count_01_02 +=1
                cond = "01_02"
                
                
            #VOC > 1.0
            #Two Way Streets
            #TransCAD Lanes
            elif ((row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and
                  (row[13] is not None and row[14] is not None and (row[10] == 0))):
                
                current_width = (row[13] + row[14])*11
                expanded_width = ((row[13] + row[14]+2)*11)
                #print("Two Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                voc_above +=1
                count_01_03 +=1
                cond = "01_03"
                
            #VOC > 1.0
            #Two Way Streets
            #ASSUMED 2 Lanes
            elif ((row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and
                  (row[13] is None and row[14] is None and (row[10] == 0) and (row[19] is not None) and row[19] < param_two_way)):
                
                current_width = 22
                expanded_width = 44
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 2
                voc_above +=1
                count_01_04 +=1
                cond = "01_04"
                
            
            #CONDITIONAL GROUP 02
            
            #VOC < 1.0
            #One Way Streets
            #TransCAD Lanes
            elif (not None in (row[17], row[18], row[13], row[14]) and (row[17] < 1.0 and row[18] < 1.0)):
        
            
# #             elif ((row[17] is not None and row[18] is not None) and (row[17] < 1.0 and row[18] < 1.0) and
# #                   (row[13] is not None and row[14] is not None) and row[10] in [1,2]):
                
                #current_width = (max(row[13],row[14]))*11
                current_width = 50
                expanded_width = current_width
                #print("One Way - VOC Max <1.0, Current:{0}, Proposed: {1} - {2}-{3}".format(current_width,expanded_width, row[17], row[18]))
                row[11] = current_width
                row[12] = expanded_width
                count_02_01 +=1
                voc_below += 1
                cond = "02_01"
                
            
            #VOC < 1.0
            #One Way Streets
            #ASSUMED One Lane
            elif ((row[17] is not None and row[18] is not None) and (row[17] < 1.0 and row[18] < 1.0) and
                  (row[13] is None and row[14] is None and row[10] in [1,2] and (row[18] is not None) and row[18] < param_one_way)):
                
                current_width = 11
                expanded_width = current_width
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 1
                count_02_02 +=1
                voc_below += 1
                cond = "02_02"
                
            #VOC < 1.0
            #Two Way Streets
            #TransCAD Lanes
            elif (not None in (row[17], row[18], row[13], row[14]) and (row[17] < 1.0 and row[18] < 1.0) and 
                  (row[10] == 0)):
            
            
#             elif ((row[17] is not None and row[18] is not None) and ((row[17] < 1.0) or (row[18] < 1.0)) and 
#                   (row[13] is not None and row[14] is not None and (row[10] == 0))):
                
                current_width = (row[13] + row[14])*11
                expanded_width = current_width
                #print("Two Way - VOC Max <1.0, Current:{0}, Proposed: {1}".format(current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                count_02_03 +=1
                voc_below += 1
                cond = "02_03"
                
            #VOC < 1.0
            #Two Way Streets
            #ASSUMED Two Lanes
            elif ((row[17] is not None and row[18] is not None) and (row[17] < 1.0 and row[18] < 1.0) and
                  (row[13] is None and row[14] is None and (row[10] == 0) and (row[19] is not None) and row[19] < param_two_way)):
                
                current_width = 22
                expanded_width = current_width
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 2
                count_02_04 +=1
                voc_below += 1
                cond = "02_04"
                    
            #CONDITIONAL GROUP 03
            
            #VOC Null
            #One Way Street
            #TransCAD Lanes
            #note this most likely does not occur but was implemented to catch records if they do occur
            elif (None in (row[17],row[18]) and (not None in (row[13],row[14]) and row[10] in [1,2])):
                                                 
                #(row[13] is not None and row[10] in [1,2]) or (row[14] is not None and row[10] in [1,2])):
                
                current_width = (max(row[13],row[14]))*11
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                row[11] = current_width
                row[12] = current_width
                count_03_01 +=1
                voc_null += 1
                cond = "03_01"
                
            #VOC Null
            #One Way Street
            #ASSUMED One Lane
            elif ((row[17] is None) or (row[18] is None) and
                  (row[13] is None and row[14] is None and row[10] in [1,2] and (row[19] is not None) and row[19] < param_one_way)):
                
                current_width = 11
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 1 lane".format(current_width))
                row[11] = current_width
                row[12] = current_width
                row[20] = 1
                count_03_02 +=1
                voc_null += 1
                cond = "03_02"
                
            #VOC Null
            #Two Way Street
            #TransCAD Lanes
            #note this most likely does not occur but was implemented to catch records if they do occur
            elif ((row[17] is None) or (row[18] is None) and
                  (row[13] is not None and row[14] is not None and (row[10] == 0))):
                
                current_width = (row[13] + row[14])*11
                #print("Two Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                row[11] = current_width
                row[12] = current_width
                count_03_03 +=1
                voc_null += 1
                cond = "03_03"
                
            #VOC Null
            #Two Way Street
            #ASSUMED Two Lanes
            elif ((row[17] is None) or (row[18] is None) and
                  (row[13] is None and row[14] is None and (row[10] == 0) and (row[19] is not None) and row[19] < param_two_way)):
                
                current_width = 22
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 2 lanes".format(current_width))
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                count_03_04 +=1
                voc_null += 1
                cond = "03_04"
                
            #VOC Null
            #Mean ROW 2020 Null
            #Street Class = A41
            #ASSUMED Two Lanes, ASSUME Modal Width (current and future) = 22'
            elif ((row[17] is None) or (row[18] is None) and
                  ((row[13] is None and row[14] is None and row[19] is None) and row[21] == 'A41')):
                
                current_width = 22
                #print("A41 Assumption - VOC Max: NULL, Current:NULL, Proposed: N/A Assumed 2 lanes".format(current_width))
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                count_03_05 +=1
                voc_null += 1
                cond = "03_05"
                                          
            #cursor.updateRow(row)
            count = count + 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))
            
            
print("Done - processed {0} records, {1} errors".format(count, err_count))
print("01-01 VOC >1.0 - One Way - TransCad Lanes: {}".format(count_01_01))
print("01-02 VOC >1.0 - One Way - Assumed Lanes: {}".format(count_01_02))
print("01-03 VOC >1.0 - Two Way - TransCad Lanes: {}".format(count_01_03))
print("01-04 VOC >1.0 - Twy Way - Assumed Lanes: {}".format(count_01_04))

print("02-01 VOC <1.0 - One Way - TransCad Lanes: {}".format(count_02_01))
print("02-02 VOC <1.0 - One Way - Assumed Lanes: {}".format(count_02_02))
print("02-03 VOC <1.0 - Two Way - TransCad Lanes: {}".format(count_02_03))
print("02-04 VOC <1.0 - Two Way - Assumed Lanes: {}".format(count_02_04))

print("03-01 VOC NULL - One Way - TransCad Lanes: {}".format(count_03_01))
print("03-02 VOC NULL - One Way - Assumed Lanes: {}".format(count_03_02))
print("03-03 VOC NULL - Two Way - TransCad Lanes: {}".format(count_03_03))
print("03-04 VOC NULL - Two Way - Assumed Lanes: {}".format(count_03_04))

print("03-05 A41 NULL -   N/A   - Assumed Lanes: {}".format(count_03_05))

print("VOC Above: {}".format(voc_above))
print("VOC Below: {}".format(voc_below))
print("VOC Null: {}".format(voc_null))
print("Modal Widths - Traffic complete.")


### Calculate Modal Widths for Bike Facilities


- Existing width is calculated into `modal_width_bike_exist` and `modal_width_bike_prop`

- Proposed width is then updated into `modal_width_bike_prop` based on proposed and redevelopment bike plan attributes.

- If a give segment has a value for existing bike facilites, but no values for proposed facilites, existing width is calculted into proposed width.


**Assumptions, Notes, and Parameters**
1. "Protected Bike Lane", "Buffered Bike Lane" = 10'
2. "Bike Lane", "Climbing Lane", "Shoulder Bikeway" = 7'


In [ ]:
#fields to expose to update cursor for bike facilities
fields = ["Fac_Type_BE",            #0
          "modal_width_bike_exist", #1
          "Fac_Type_BP",            #2
          "Fac_Type_BR",            #3
          "modal_width_bike_prop"]  #4

#update bike existing width columns
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite_05a,fields,'NOT ("Fac_Type_BE" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike existing calculations
            if row[0] == "Protected Bike Lane" or row[0] == "Buffered Bike Lane":
                row[1] = 12
                row[4] = 12
                    
            elif (row[0] == "Bike Lane" or
                  row[0] == "Climbing Lane" or
                  row[0] == "Shoulder Bikeway" or
                  row[0] == "Shared Use Path" or
                  row[0] == "Shared Roadway"):
                row[1] = 7
                row[4] = 7
                    
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike existing width calculated\n")

#perform second pass w/ update cursor for bike proposed features (will over write previous calcs if there is an improvement)
#update bike proposed width columns
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite_05a,fields,'NOT ("Fac_Type_BP" IS NULL AND "Fac_Type_BR" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike proposed & redev calculations
            if (row[2] == "Protected Bike Lane" or
                  row[2] == "Buffered Bike Lane" or
                  row[3] == "Protected Bike Lane" or
                  row[3] == "Buffered Bike Lane"):
                row[4] = 12
                
            elif (row[2] == "Bike Lane" or
                  row[2] == "Climbing Lane" or
                  row[2] == "Shoulder Bikeway" or
                  row[2] == "Shared Use Path" or
                  row[2] == "Shared Roadway" or
                  row[3] == "Bike Lane" or
                  row[3] == "Climbing Lane" or
                  row[3] == "Shoulder Bikeway" or
                  row[3] == "Shared Use Path" or
                  row[3] == "Shared Roadway"):
                row[4] = 7
                
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike proposed width calculated\n")
print("Modal Widths - Bike complete.")

In [ ]:
#data analysis for medians
fields = ["SEGMENTID",
          "FULLNAME",
          "mean_row_2020",
          "NBRightP_1",
          "SBLeftPe_1",
          "FID_Ped_Improve_temp",
          "FID_Ped_Add_temp",
          "modal_width_ped_exist",
          "median_type",
          "median_width"]

fields_median = ["SEGMENTID",
          "FULLNAME",
          "mean_row_2020",
          "modal_width_medians",
          "median_type",
          "median_width"]

#two way streets, null lane count, row > 100
filt2 = ((~sdf["FID_Ped_Improve_temp"].isnull()) |
         (~sdf["FID_Ped_Add_temp"].isnull())
        )


filt3 = ((sdf["FID_Ped_Improve_temp"].isnull()) &
         (~sdf["FID_Ped_Add_temp"].isnull()) &
         (sdf["NBRightP_1"] == 0) &
         (sdf["SBLeftPe_1"] == 0)
        )
         
#one way streets, null lane count, row > 75
filt4 = ((sdf["AB_LANE"].isnull()) &
         (sdf["BA_LANE"].isnull()) &
         (sdf["ONEWAY"].isin([1,2])) &
         (sdf["mean_row_2020"]>75)
        )

#two way streets, null lane count, row > 100
filt_median = ~sdf["median_type"].isnull()

#pd.set_option('display.max_rows', 250)
sdf.loc[filt_median,fields_median].sort_values(by="FULLNAME")

In [ ]:
sdf2.loc[sdf2["median_type"]== "R","median_type"] = "Raised with Curb"
sdf2.loc[sdf2["median_type"]== "F","median_type"] = "Flush Two-Way Left Turn"
sdf2.loc[sdf2["median_type"]== "C","median_type"] = "Composite"
sdf2.loc[sdf2["median_type"]== "FB","median_type"] = "Flush with Barrier"

sdf2.loc[sdf2["median_width"]== "A","median_width"] = 5
sdf2.loc[sdf2["median_width"]== "B","median_width"] = 11
sdf2.loc[sdf2["median_width"]== "C","median_width"] = 17
sdf2.loc[sdf2["median_width"]== "D","median_width"] = 22

sdf2_filt = ((~sdf2["median_type"].isnull()) &
             (~sdf2["median_type"].isin(["Raised", "Flush", "Composite", "Flush with Barrier"]))
            )

sdf2.loc[~sdf2["median_type"].isnull(),["SEGMENTID", "median_type", "median_width"]]

In [ ]:
sdf_fields

### Calculate Modal Widths for Medians

- Median data was manually classified by reviewing street segments where `mean_row_2020 > 75`, then estimaing median type and median width using aerial imagery.
- Width estimates were then mapped into `modal_width_medians` (see assumptions below)

**Assumptions, Notes, and Parameters**
1. Median width esitmations were mapped to modal with values as follows:

|Median Width Estimate| Modal Width Value  |
|:--|:--|
| < 5' |5'  |
| 5' - 10' |11' (increase by 1' to match avg. lane width)  |
| 15' - 20' |17'  |
| >20' |22'  |


In [ ]:
#Calculate Modal Width for Medians

# "A","median_width"] = 5
# "B","median_width"] = 11
# "C","median_width"] = 17
# "D","median_width"] = 22

#fields to expose to update cursor
fields = ["median_type",         #0
          "median_width",        #1
          "modal_width_medians"] #2

#update median width columns
print("Update rows for Medians widths..")
count = 0
with arcpy.da.UpdateCursor(modal_composite,fields,'NOT ("median_type" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        
        # 6/16/2021 - increase all values by 2
        
        try:
            if row[1] == "A":
                row[2] = 7
                count +=1
            elif row[1] == "B":
                row[2] = 13
                count +=1
            elif row[1] == "C":
                row[2] = 19
                count +=1
            elif row[1] == "D":
                row[2] = 14
                count +=1
                    
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            
print("median widths calculated for {} items".format(count))
print("Modal Widths - Medians complete.")

### Calculate Modal Widths for Ped Facilities

- Existing width is calculated into `modal_width_ped_exist` and `modal_width_ped_prop`

- Proposed width `modal_width_ped_prop` is then updated if a segment had no ped facilites, was not listed in ped facilites, or was listed in "Candidate Walkways" or "Candidate Upgrades", and the proposed width was greater than the existing width. (see assumptions below)


**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Any segment listed in `NewTier` == 4 is set to 8' if oneway, 16' if two-way for future widths.
3. The following steet segments types are set to 12' for `modal_width_ped_prop`
    1. Any street segment listed in "Candidate Walkways", and "Candidate Upgrades" datasets
    2. Any street segment listed in Pedestrian Sidewalk Inventory as having no facilites, but are not listed in "Candidate Walkways" or "Candidate Upgrades"
    3. Any street segment NOT listed in Pedestrian Sidewalk Inventory, AND not listed in "Candidate Walkways", AND not listed in "Candidate Upgrades", with `CLASS = A41`


In [ ]:
#calculate modal width for ped facilities

#assumptions based on ped data dictionary values for PED ZONE WIDTH
# greater than 10' = 12' DOMAIN 04
# 5'-10' (clear and obstruced types) = 8' DOMAIN 03, DOMAIN 02
# less than 5' = 4' DOMAIN 01

p1 = 12
p2 = 8
p3 = 4

#facilities combinations
#24' total
#20' total
#16' total
#12' total
#8' total
#4' total
# none on either side

#existing ped facilities

#expose fields to update cursor
ped_e_fields = ["SEGMENTID",             #0 - SegmentID
                "NBRightP_1",            #1 - Pedestrian Zone Width (NB)
                "SBLeftPe_1",            #2 - Pedestiran Zone Width (SB)
                "modal_width_ped_exist", #3 - Modal Width - Ped (Existing)
                "modal_width_ped_prop",  #4 - Modal Width - Ped (Proposed)
                "mean_row_2020",         #5 - Mean ROW Analysis
                "FID_Ped_Improve_temp",  #6 - Ped Improvements
                "FID_Ped_Add_temp",      #7 - Ped Additions
                "CLASS",                 #8 - Street Class (tiger)
                "NewTier",               #9 - Bus Transit - New Proposed Tier
                "ONEWAY",                #10 - Bus Transit - New Proposed Tier
                "NBRightP_7",            #11 - Pedestrian Buffer Roadway Buffer (NB)
                "SBLeftPe_6"]            #12 - Pedestrian Buffer Roadway Buffer (SB)


count = 0
err_count = 0

print("Update rows with Ped Zone widths..")

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields) as cursor:
# with arcpy.da.UpdateCursor(modal_composite_05a, ped_e_fields,'NOT ("NBRightP_1" IS NULL OR "SBLeftPe_1" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #24' total
            count +=1
            if row[1] == 4 and row[2] == 4:
                width = p1*2
                row[3] = width
                row[4] = width
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
        
            #20' total
            elif ((row[1] == 4 and row[2] in [2,3])or
                  (row[1] in [2,3] and row[2] == 4)):
                width = p1+p2
                row[3] = width
                row[4] = width
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))

            #16' total
            elif ((row[1] in [2,3]) and (row[2] in [2,3])):
                width = p2*2
                row[3] = width
                row[4] = width
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))

            #12' total
            elif ((row[1] == 1 and (row[2] in [2,3]))or
                  ((row[1] in [2,3]) and row[2] == 1)):
                width = p2+p3
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
            
            #8' total
            elif row[1] == 1 and row[2] == 1:
                width = p3*2
                row[3] = width
                row[4] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))

            #4' total
            elif ((row[1] == 1 and row[2] == 0) or
                  (row[1] == 0 and row[2] == 1)):
                width = p3
                row[3] = width
                row[4] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))


            # none on either side, set future width to 12'
            elif (row[1] == 0 and row[2] == 0):
                width = 0
                row[3] = width
                row[4] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                  
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Done - processed {0} records, {1} errors".format(count, err_count))


#assumptions based on ped data dictionary values for PED FURNITURE ZONE WIDTH
# greater than 4' = 6' DOMAIN 03
# 0' to 4' = 4' DOMAIN 02
# 0' = 0' DOMAIN 01

#add cursor pass for furniture zone

print("Update rows with Ped Furnitur widths..")

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            #12' total
            count +=1
            if row[11] == 3 and row[12] == 3:

                row[3] += 12
                row[4] += 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
        
            #10' total
            elif ((row[11] == 3 and row[12] == 2)or
                  (row[11] == 2 and row[12] == 3)):
                
                row[3] += 10
                row[4] += 10
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
            #8' total
            elif (row[11] == 2 and row[12] == 2):
         
                row[3] += 8
                row[4] += 8
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
            #4' total
            elif ((row[11] == 2 and row[12] == 1)or
                  (row[11] == 1 and row[12] == 2)):
        
                row[3] += 4
                row[4] += 4
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))


                  
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Done - processed {0} records, {1} errors".format(count, err_count))
                
                

#proposed ped facilities - use a second pass with update cursor to update proposed widths to 12' if they
# have segments listed in Ped improvement datasets, and the existing width is less than 12'
count = 0
err_count = 0

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields,'(NOT "FID_Ped_Improve_temp" IS NULL) OR (NOT "FID_Ped_Add_temp" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #only update proposed width if is less than the current width
            if ((row[4] is not None) and row[4] < 12):
                row[4] = 12
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
#proposed bus transit tier 4 - update width to 16' if twoway, 8' if oneway
count = 0
err_count = 0

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields,'"NewTier" = 4') as cursor:
    for row in cursor:
        #print("New Tier = 4 Row")
        try:
            #only update proposed width if is less than the current width
            if (((row[10] is not None) and row[10] == 0) and
                ((row[4] is None) or (row[4] is not None) and row[4] < 16)):
                row[4] = 16
                #print("Tier4 two way")
                
            elif((row[10] is not None) and row[10] in [1,2] and
                 ((row[4] is None) or (row[4] is not None) and row[4] < 8)):
                row[4] = 8
                #print("Tier4 one way")             
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

# perform final pass with update cursor to add proposed width to facilites that have no sidewalks
# and are not covered by sidewalk inventory, but are residential (class = 'A41')
count = 0
err_count = 0

#with arcpy.da.UpdateCursor(modal_composite_05a, ped_e_fields,'"NBRightP_1" IS NULL AND "FID_Ped_Improve_temp" IS NULL AND "FID_Ped_Add_temp" IS NULL AND "CLASS" = \'A41\' ') as cursor:
with arcpy.da.UpdateCursor(modal_composite, ped_e_fields,'"NBRightP_1" IS NULL AND "FID_Ped_Improve_temp" IS NULL AND "FID_Ped_Add_temp" IS NULL') as cursor:
    for row in cursor:
        #print("final pass")
        try:
            #only update proposed width if is less than the current width
            if (row[8] == 'A41' and ((row[4] is not None and row[4] < 12) or row[4] is None)):
            #if ((row[8] == 'A41') and (row[4] is None)):
                row[4] = 12   
                #print("pass 3 for A41")
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

print("Modal Widths - Ped complete.") 

### Calculate Modal Widths for on-street Parking segments

- Existing width of 7' is calculated into `modal_width_park_exist` and `modal_width_park_prop` for the following segment types:
    - Roadway Buffer = "Yes" `NBRightP_7 == 1 OR SBLeftPed_6 == 1`
    - Pedestrian Zone Intrusion = "Yes" `NBRightP_4 == 1 or SBLeftPed_4`
    
- If a segment is not listed in the Pedestrian Plan Sidewalk inventory, and it's street type is residential (A41), and mean_row_2020 - modal_width_auto_exist > 22, on street parking is assumed on both sides.




**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Street segments listed in "Candidate Walkways", and "Candidate Upgrades" datasets are assumed to be 6' sidewalk on both sides, (12' total)
4. Street segmnents listed in Pedestrian Sidewalk Inventory having no facilites, but are not listed in "Candidate Walkways", or "Candidate Upgrades" and are PLACEHOLDER are assumed to be upgraded to one 6' sidewalk. 6' total.


In [ ]:
#calculate modal width for street segements with on-street parking facilities


#expose fields to update cursor
parking_fields = ["SEGMENTID",              #0 - SegmentID
                  "NBRightP_7",             #1 - Road Buffer (NB)
                  "SBLeftPe_6",             #2 - Road Buffer (SB)
                  "NBRightP_4",             #3 - Ped Intrusion (NB)
                  "SBLeftPe_4",             #4 - Ped Intrusion (SB)
                  "modal_width_park_exist", #5 - Modal Width - Park (Existing)
                  "modal_width_park_prop",  #6 - Modal Width - Park (Proposed)
                  "mean_row_2020",          #7 - Mean ROW Analysis
                  "modal_width_auto_exist", #8 - Modal Width - Auto (Existing)
                  "CLASS"]                  #9 - Street Class (tiger)

park_ped_count = 0
park_assume_count = 0
err_count = 0

param = 22

print("Update rows for Parking widths..")

#calculate parking width for segments based on ped inventory facilities
with arcpy.da.UpdateCursor(modal_composite_05a, parking_fields,'"NBRightP_7" IS NOT NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[1] == 1 or row[2] == 1 or row[3] == 1 or row[4] == 1:
                row[5] = 8
                row[6] = 8
                park_ped_count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

#calculate parking width for segments not listed in ped inventory facilities
with arcpy.da.UpdateCursor(modal_composite_05a, parking_fields,'"NBRightP_7" IS NULL AND "CLASS" = \'A41\'') as cursor:
    for row in cursor:
        #print("row")
        try:
            if (row[7] is not None) and (row[8] is not None) and ((row[7] - row[8]) > param):
                row[5] = 8
                row[6] = 8
                park_assume_count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Parking widths (ped) calculated for {0} items".format(park_ped_count))
print("Parking widths (assumed) calculated for {0} items".format(park_assume_count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Parking complete.")

### Calculate Modal Widths for Bus Transit segments

TEXT TO BE REPLACED

- Existing width of 7' is calculated into `modal_width_park_exist` and `modal_width_park_prop` for the following segment types:
    - Roadway Buffer = "Yes" `NBRightP_7 == 1 OR SBLeftPed_6 == 1`
    - Pedestrian Zone Intrusion = "Yes" `NBRightP_4 == 1 or SBLeftPed_4`
    
- If a segment is not listed in the Pedestrian Plan Sidewalk inventory, and it's street type is residential (A41), and mean_row_2020 - modal_width_auto_exist > 22, on street parking is assumed on both sides.




**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Street segments listed in "Candidate Walkways", and "Candidate Upgrades" datasets are assumed to be 6' sidewalk on both sides, (12' total)
4. Street segmnents listed in Pedestrian Sidewalk Inventory having no facilites, but are not listed in "Candidate Walkways", or "Candidate Upgrades" and are PLACEHOLDER are assumed to be upgraded to one 6' sidewalk. 6' total.


In [ ]:
#calculate modal width for bus facilities


#expose fields to update cursor
bus_fields = ["SEGMENTID",              #0 - SegmentID
              "NAME",                   #1 - Name
              "modal_width_bus_exist",  #2 - Modal Width - Bus (Existing)
              "modal_width_bus_prop",   #3 - Modal Width - Bus (Proposed)
              "mean_row_2020",          #4 - Mean ROW Analysis
              "CLASS",                  #5 - Street Class (tiger)
              "NewTier",                #6 - Bus Transit - New Tier
              "ONEWAY"]                 #7 - OneWay

bus_count = 0
new_count = 0
err_count = 0

#existing bus facilites by SegmentID (replace with Bus Plan data when available)

IDs_exist = (
    18813,18858,18867,18914,18968,18984,19024,19033,19068,19090,19140,19148,19148,19234,19272,
    19306,19320,19333,19357,19401,19441,19501,19595,19672,19758,19790,21993,22030,22049,22112,
    22163,22169,33146)

where_clause = '"SEGMENTID" IN {}'.format(IDs_exist)

print("Update rows for Bus widths..")
print("Calculating existing...")
#calculate existing Priority Bus facilities
with arcpy.da.UpdateCursor(modal_composite_05a, bus_fields, '"SEGMENTID" IN {}'.format(IDs_exist)) as cursor:   
    for row in cursor:
        #print("row")
        try:
            if row[1] == "HOTEL":
                row[2] = 24
                row[3] = 24
                bus_count += 1
            
            elif row[1] == "KALAKAUA" or row[1] == "KING":
                row[2] = 12
                row[3] = 12
                bus_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Existing calc done.")
            
#calculate proposed Tier4 Bus facilities
print("Calculating proposed...")
with arcpy.da.UpdateCursor(modal_composite_05a, bus_fields, '"NewTier" = 4') as cursor:
    for row in cursor:
        try:
            if ((row[7] is not None) and row[7] == 0):
                row[3] = 24
                new_count += 1
            elif((row[7] is not None) and row[7] in [1,2]):
                row[3] = 12
                new_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
print("Proposed calc done.") 
            
print("Bus widths - Existing -  calculated for {0} segments".format(bus_count))
print("Bus widths - New -       calculated for {0} segments".format(new_count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Bus complete.")

### Calculate Unconstrained Modal Width

Use previously calculated modal widths for each modal type to generate a total unconstrained modal width value.


**Assumptions, Notes, and Parameters**
1. Future Tier 4 Bus Priority lanes are ignored in the unconstrained width calcuation based on the assumption that Bus Priority lanes will always be placed in existing auto lanes.
2. Do not sum Ped Width and Bike Width if future Bike Type = Shared Use Path
3. Ignore all bike facility types of shared road way in calculation



In [ ]:
#calcualte unconstrained modal widths

fields = ["SEGMENTID",               #0
          "modal_width_ped_prop",    #1
          "modal_width_bike_prop",   #2
          "modal_width_auto_prop",   #3
          "modal_width_bus_prop",    #4
          "modal_width_park_prop",   #5
          "modal_width_medians",     #6
          "modal_width_max",         #7
          "Fac_Type_BP",             #8
          "Fac_Type_BR",             #9
          "NewTier"]                  #10        

count = 0
err_count = 0

#desc

print("Calculating width..")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #igonore shared road-way widths from bike width
            if row[8] == "Shared Roadway" or row[9] == "Shared Roadway":     
                row[7] = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                count +=1
                
            #ignore bike shared use path for ped > 7    
            elif (row[8] == "Shared Use Path" or row[9] == "Shared Use Path") and row[1] is not None and row[1] > 7:
                row[7] = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                count +=1
                
            #if shared use path is proposed on segment where future ped width is less than 7, assume 7 for ped width in calc   
            elif (row[8] == "Shared Use Path" or row[9] == "Shared Use Path") and row[1] is not None and row[1] < 7:
                row[7] = sum(filter(None,(row[1], 7,  row[3], row[4], row[5], row[6])))
                count +=1
            
            #summarize all other segments
            else:
                row[7] = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
                count +=1
                
            #row[7] = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
            #row[7] = sum(filter(None,(row[1], row[2], row[3], row[5], row[6])))
            
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Unconstrained widths calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Unconstrained complete.")


In [ ]:
#add additonal fields for modal width/area metrics

#length_ft - to capture line length in ft prior to conversion to WebMercator. Calculated from shape_length
#mean_row_area - estimated area, 2020 row estimate X length
#modal_area_max - estimated unconstrained area, unconstrained width x length
#modal_area_diff - difference between Unconstrained Area and 2020 Estimate. Uncon - 2020. Positive values will represent exceedences, negative will be surplus 


print("adding fields")
arcpy.AddField_management(modal_composite_05a,field_name="length_ft",field_alias = "Length (ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="mean_row_area",field_alias = "ROW Area Estimate (sq. ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="modal_area_max",field_alias = "Unconstrained Modal Area (sq. ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="modal_area_diff",field_alias = "Unconstrained Modal Area Exceedence (sq. ft.)",field_type="FLOAT")
print("done")

In [ ]:
list_fields = arcpy.ListFields(modal_composite_05a)
field_names = [i.name for i in list_fields]

field_check = "modal_area_diff"

if field_check in field_names:
    print("field already exists")
else:
    print("not found")
    

In [ ]:
#calculate modal area fields

fields = ["SEGMENTID",       #0
          "modal_width_max", #1
          "Shape_Length",    #2
          "length_ft",       #3
          "mean_row_area",   #4
          "modal_area_max",  #5
          "modal_area_diff", #6
          "mean_row_2020"]   #7
   

count = 0
err_count = 0

#desc

print("Calculating areas..")

with arcpy.da.UpdateCursor(modal_composite, fields, 'NOT "mean_row_2020" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            length_ft = row[2]
            mean_row_area = length_ft * row[7]
            modal_area_max = length_ft * row[1]
            modal_area_diff = modal_area_max - mean_row_area
            
            row[3] = length_ft
            row[4] = mean_row_area
            row[5] = modal_area_max
            row[6] = modal_area_diff
            
            count +=1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))

print("Modal Areas - complete.")

In [ ]:
#reset all (or some) width and area fields to null (use during development)

fields = ["modal_width_ped_exist",
          "modal_width_ped_prop",
          "modal_width_bike_exist",
          "modal_width_bike_prop",
          "modal_width_auto_exist",
          "modal_width_auto_prop",
          "modal_width_bus_exist",
          "modal_width_bus_prop",
          "modal_width_park_exist",
          "modal_width_park_prop",
          "modal_width_medians",
          "modal_width_max",
          "mean_row_area",
          "modal_area_max",
          "modal_area_diff"]

err_count = 0

print("Reseting modal widths and areas to Null...")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            
#             row[0] = None    # modal_width_ped_exist
#             row[1] = None    # modal_width_ped_prop
#             row[2] = None    # modal_width_bike_exist
#             row[3] = None    # modal_width_bike_prop
#             row[4] = None    # modal_width_auto_exist
#             row[5] = None    # modal_width_auto_prop
#             row[6] = None    # modal_width_bus_exist
#             row[7] = None    # modal_width_bus_prop
#             row[8] = None    # modal_width_park_exist
#             row[9] = None    # modal_width_park_prop
#             row[10] = None   # modal_width_medians
            row[11] = None   # modal_width_max
            row[12] = None   # modal_width_park_prop
            row[13] = None   # modal_width_medians
            row[14] = None   # modal_width_max




            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print(error)

print("All widths and areas reset to Null.")
            

In [ ]:

##OLD
#fields to expose to update cursor for bike facilities
fields = ["Fac_Type_BE","modal_width_bike_exist","Fac_Type_BP","Fac_Type_BR","modal_width_bike_prop"]

#update bike width column
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite_05a,fields,'NOT ("Fac_Type_BE" IS NULL OR "Fac_Type_BP" IS NULL OR "Fac_Type_BR" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike existing calculations
            if row[0] == "Protected Bike Lane" or row[0] == "Buffered Bike Lane":
                row[1] = 10
                
            elif row[0] == "Bike Lane" or row[0] == "Climbing Lane" or row[0] == "Shoulder Bikeway":
                row[1] = 7
            
            #bike proposed & redev calculations
            elif (row[2] == "Protected Bike Lane" or
                  row[2] == "Buffered Bike Lane" or
                  row[3] == "Protected Bike Lane" or
                  row[3] == "Buffered Bike Lane"):
                row[4] = 10
                
            elif (row[2] == "Bike Lane" or
                  row[2] == "Climbing Lane" or
                  row[2] == "Shoulder Bikeway" or
                  row[3] == "Bike Lane" or
                  row[3] == "Climbing Lane" or
                  row[3] == "Shoulder Bikeway"):
                row[4] = 7
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike data width calculated\n")
            
#update ln_prop_num w/ update cursor
print("parse ln_prop to int")
with arcpy.da.UpdateCursor(modal_composite_04a,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[1] == '5+ bus':
                row[5] = 5
            else:
                row[5]= int(row[1])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("ln_prop parsed\n")
            
#update row_exist_num w/ update cursor
print("parse row_exist to num")
with arcpy.da.UpdateCursor(modal_composite_04a,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[2] == 'Var. to 80':
                row[6] = 80
            elif row[2] == 'Var. 60-70':
                row[6] = 65
            elif row[2] == 'Var. 36-50':
                row[6] == 61
            elif row[2] == 'Var. 14-26':
                row[6] = 20
            elif row[2] == '86/100':
                row[6] = 93
            elif row[2] == '60-90':
                row[6] = 75
            elif row[2] == '60-76':
                row[6] = 68
            elif row[2] == '60-70':
                row[6] = 65
            elif row[2] == '60-64':
                row[6] = 62
            elif row[2] == '50-80':
                row[6] = 65
            elif row[2] == '50-60':
                row[6] = 55
            elif row[2] == '44-56':
                row[6] = 50
            elif row[2] == '40/50':
                row[6] = 45
            elif row[2] == '40-56':
                row[6] = 48
            elif row[2] == '30-50':
                row[6] = 40
            elif row[2] == '30-40':
                row[6] = 35
            elif row[2] == '25-50':
                row[6] = 38
            elif row[2] == '20-40':
                row[6] = 30
            elif row[2] == '25-50':
                row[6] = 38
            elif row[2] == '120-140':
                row[6] = 130
            elif row[2] == '113.5':
                row[6] = 114
            elif row[2] == '100-110':
                row[6] = 105
            elif row[2] == '0':
                row[6] = None
            else:
                row[6]= int(row[2])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("row_exist parsed\n")            
                  
#update row_prop_num w/ update cursor
print("parse row_prop to int")
with arcpy.da.UpdateCursor(modal_composite_04a,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:          
            if row[3] == 'Var. to 80':
                row[7] = 80
            elif row[3] == 'Var. 14-26':
                row[7] = 20
            elif row[3] == '86/100':
                row[7] = 93
            elif row[3] == '60-90':
                row[7] = 75
            elif row[3] == '60-76':
                row[7] = 68
            elif row[3] == '60-70':
                row[7] = 65
            elif row[3] == '60-64':
                row[7] = 62
            elif row[3] == '56-80':
                row[7] = 68
            elif row[3] == '50-60':
                row[7] = 55
            elif row[3] == '40/50':
                row[7] = 45
            elif row[3] == '30-40':
                row[7] = 35
            elif row[3] == '25-50':
                row[7] = 38
            elif row[3] == '120-140':
                row[7] = 130
            elif row[3] == '113.5':
                row[7] = 114
            elif row[3] == '100-110':
                row[7] = 105
            else:
                row[7] = int(row[3])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("row_prop parsed\n")

print("Processing Complete")